

Ever since I started studying machine learning, I understood that I also needed to know statistics, analyse and visualize data. Over time I analyzed data, used functions from libraries for statistical analysis and for creating machine learning models. I was interested in knowing both, to work with different libraries in order to get meaning from data.

The purpose of this analysis was to observe what are the most important characteristics that define those who have the job of Data Scientist or Machine Learning Engineer. The second purpose was to analise data by taking into account each important characteristic. 

First, I analysed the data in order to observ it.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
data=pd.read_csv("../input/kaggle-survey-2020/kaggle_survey_2020_responses.csv", low_memory=False)
data.drop([0], axis = 0, inplace = True)

In [ ]:
data.describe()

It seems that the top is: Python, India, Student,and in the end OTHER and None. Next, I wanted to observ jobs' distribution.

In [ ]:
no_roles=[]

def how_many(df,vals):
  k=0
  #print(vals)
  for j in vals:    
    for i in range(len(df)):
      if df[i]==j:
        k=k+1
      #print(f"i={i}, j={j}, k={k}")
    no_roles.append(k)    
    k=0
  return no_roles

vals=data['Q5'].unique()
no_of_roles=how_many(data['Q5'].values,vals)
df = pd.DataFrame(no_of_roles, columns=['Number'])
df.insert(1,"Role",vals)

In [ ]:
fig = px.pie(df, values='Number', names='Role', title='Kagglers role distribution', color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

Data Scientists represent 13.9% from the total number of Kagglers survey respondents, and Machine Learning Engineers represent 5.6% from the total. Next, I filtered the initial dataset in order to obtain records related only to Data Scientist and Machine Learning Engineer jobs.

In [ ]:
data_A=data
data = data.loc[lambda x: x['Q5'].isin(['Data Scientist','Machine Learning Engineer']), :]

I wanted to see if the responses could help train a classification neural network model. If this model has a good accuracy, I can further analyze the most important features that differentiate Data Scientists from Machine Learning Engineers. I placed all the dataframe columns in a list of features, except for the target Q5 (The current role) and the time spent by each respondent to answer the survey (I thought it is not that important).

In [ ]:
features=data.columns.to_list()
type(features)
features.remove('Q5')
features.remove('Time from Start to Finish (seconds)')

I declared the target (dependent variable) and the input (independent variables).

In [ ]:
y=data.Q5.map({'Data Scientist':0,'Machine Learning Engineer':1})
X=data[features]
X=X.replace(np.nan,0)
X = X.astype(str)

As all the variables are categorical, I prepared the training and the testing datasets and encoded the data.

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=0)
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
def prepare_inputs(X_train, X_test):
	oe = OrdinalEncoder()
	oe.fit(X_train)
	X_train_enc = oe.transform(X_train)
	X_test_enc = oe.transform(X_test)
	return X_train_enc, X_test_enc
 
# prepare target
def prepare_targets(y_train, y_test):
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc
 
# feature selection
def select_features(X_train, y_train, X_test):
	fs = SelectKBest(score_func=chi2, k='all')
	fs.fit(X_train, y_train)
	X_train_fs = fs.transform(X_train)
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs, fs

In [ ]:
X_train_enc, X_val_enc = prepare_inputs(X_train, X_val)
y_train_enc, y_val_enc = prepare_targets(y_train, y_val)

In [ ]:
import tensorflow as tf
ann=tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units=170, activation='relu'))
ann.add(tf.keras.layers.Dense(units=170,activation='relu'))
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train_enc, y_train_enc, batch_size = 35, epochs = 100)

y_pred=ann.predict(X_val_enc)
y_pred=(y_pred>0.5)

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
acc=accuracy_score(y_val_enc,y_pred)
print(acc)

This is a very good accuracy, given the fact that we took almost all the columns into consideration. Let's see the classification report.

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print('accuracy= ',accuracy_score(y_val_enc,y_pred))
class_r=classification_report(y_val_enc,y_pred)
cm=confusion_matrix(y_val_enc,y_pred)
print(class_r)
print(cm)

Next I was interested to find what are the most important features.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, chi2
def select_features(X_train, y_train, X_test):
	fs = SelectKBest(score_func=chi2, k='all')
	fs.fit(X_train, y_train)
	X_train_fs = fs.transform(X_train)
	X_test_fs = fs.transform(X_test)
	return X_train_fs, X_test_fs, fs
X_train_fs, X_val_fs, fs = select_features(X_train_enc, y_train_enc, X_val_enc)
# what are scores for the features
#for i in range(len(fs.scores_)):
	#print('Feature %d: %f' % (i, fs.scores_[i]))
top_features = sorted(zip(X_train.columns, fs.scores_), key=lambda x: x[1], reverse=True)
for i in range(0,40):
    print(top_features[i])
#print(top_features)


For now, we noticed that Q24, Q32, Q7_Part_2, Q18_Part_3, Q30, Q1 and many other features are important in determing if a Kaggler is a Data Scientist or a Machine Learning Engineer. Let's see some plots. 

We took the features in the order of their importance, as we determined above. Therefore, the next section is:

**Specific characteristics of Data Scientists and ML Engineers**

We considered Q24- yearly compensation (this is the first predictor) 

In [ ]:
ax= pd.crosstab(data['Q5'], data['Q24'],normalize='index',values='Q5',aggfunc='count')
display(ax)
fig = go.Figure(data=[
    go.Bar(name='Data Scientist',x=ax.columns,y=ax.values[0,:]),
    go.Bar(name='ML Engineers', x=ax.columns, y=ax.values[1,:])
])
fig.update_layout(barmode='group')
fig.update_layout(title_text='DS and ML Engineers Yearly Compensation- percent',title_x=0.5,xaxis={'categoryorder':'category ascending'})
fig.show()


1. There is a difference between Data Scientists and Machine Learning Engineers at the middle level of the yearly compensation interval. It seems that there are more Data Scientists that earn an amount greater than 100000 USD than Data Scientists that earn a total amount of up to 50000 USD. Machine Learning Engineers are not in the same situation. The number of them who earn a total between 100000 and 124999 USD is almost equal with the total number of those who earn between 40000 and 50000 USD.

2. Further, Data Scientists and Machine Learning Engineers are studied from the perspective of Business Intelligence Tools that they use, because this is the second predictor.


In [ ]:
ax= pd.crosstab(data['Q5'], data['Q32'],normalize='index',values='Q5',aggfunc='count')
display(ax)
fig = go.Figure(data=[
    go.Bar(name='Data Scientist',x=ax.columns,y=ax.values[0,:]),
    go.Bar(name='ML Engineers', x=ax.columns, y=ax.values[1,:])
])
fig.update_layout(barmode='group')
fig.update_layout(title_text='Business Intelligence tools preferences- percent',title_x=0.5,xaxis={'categoryorder':'category ascending'})
fig.show()

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q7_Part_2'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

#ax.plot(kind='bar', stacked=True, title = 'R usage distribution among DS and ML Eng', mark_right = True)
fig = go.Figure(data=[
    go.Bar(name='R usage', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='R usage distribution among DS and ML Eng',title_x=0.5)
fig.show()

There are some Machine Learning Engineers who use R, but in a lesser amount than Data Scientists. In fact, it is obvious that "using R" is a good predictor for classifying between ML Engineers and Data Scientists.
Do Kagglers also process images? The next predictor is Q18_Part_3 (Which categories of computer vision methods do you use on a regular basis?  (Select all that apply) - Selected Choice - Object detection methods (YOLOv3, RetinaNet, etc). Let's see the results.

In [ ]:
dataF = data_A.groupby("Q5").count()

fig = px.pie(dataF, values='Q18_Part_3', names=dataF.index.values, title='Kagglers that use Object detection methods', color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q18_Part_3'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Object detection methods- YOLOv3, RetinaNet, etc', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Object detection methods (YOLOv3, RetinaNet, etc)',title_x=0.5)
fig.show()


Data Scientists and Machine Learning Engineers differ when we talk about object detection methods usage. Machine Learning Engineers use these methods in a greater proportion than Data Scientists.
We further analyzed the data in terms of Big Data products usage.

In [ ]:
ax= pd.crosstab(data['Q5'], data['Q30'],normalize='index',values='Q5',aggfunc='count')
display(ax)
fig = go.Figure(data=[
    go.Bar(name='Data Scientist',x=ax.columns,y=ax.values[0,:]),
    go.Bar(name='ML Engineers', x=ax.columns, y=ax.values[1,:])
])
fig.update_layout(barmode='group')
fig.update_layout(title_text='Big Data products usage- percent',title_x=0.5)
fig.show()

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q1'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)
fig = go.Figure(data=[
    go.Bar(name='Data Scientist',x=ax.columns,y=ax.values[0,:]),
    go.Bar(name='ML Engineers', x=ax.columns, y=ax.values[1,:])
])
fig.update_layout(barmode='group')
fig.update_layout(title_text='DS and ML Engineers by age distribution- - percent',title_x=0.5)
fig.show()

Data Scientists age distribution is not that left sided as ML Engineers' age distribution is. It seems that the intervals of age between 18 and 21 years old and 25 and 29 years old are characterized by the presence of more ML engineers (I calculated the number as the fraction from the total numer of ML Engineers).


In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q23_Part_1'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Analyze and understand data to influence product or business decisions', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Analyze and understand data to influence product or business decisions',title_x=0.5)
fig.show()

Data Scientists definitely analyze data to influence product or business decisions if we compare them with ML Engineers.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q7_Part_3'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='SQL usage', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='SQL usage',title_x=0.5)
fig.show()

Data Scientists use SQL more than ML Engineers.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q12_Part_3'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

#ax.plot(kind='bar', stacked=True, title = 'Specialized hardware- NONE', mark_right = True)
fig = go.Figure(data=[
    go.Bar(name='Specialized hardware- NONE', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Specialized hardware- NONE',title_x=0.5)
fig.show()

A significant proportion of DS don't use a specialized hardware. For what do Data Scientists use specialized hardware?

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q18_Part_4'].fillna('unfilled'),values='Q5',normalize='index',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Image classification and other general purpose networks', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Image classification and other general purpose networks',title_x=0.5)
fig.show()

Probably for image classification (this was the next predictor). Although, we noticed that ML Engineers classify images in a greater proportion than that of the Data Scientists.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q7_Part_5'].fillna('unfilled'),values='Q5',normalize='index',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='C++ usage', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='C++ usage',title_x=0.5)
fig.show()

ML Engineers use C++ in a greater proportion than Data Scientists.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q35_A_Part_10'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

#ax.plot(kind='bar', stacked=True, title = 'Tools to help manage machine learning experiments-NONE', mark_right = True)
fig = go.Figure(data=[
    go.Bar(name='Tools to help manage machine learning experiments-NONE', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Tools to help manage machine learning experiments-NONE',title_x=0.5)
fig.show()

DS do not use any tool to help manage machine learning experiments in a greater proportion than ML Engineers.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q17_Part_7'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Convolutional Neural Networks', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Convolutional Neural Networks',title_x=0.5)
fig.show()

ML Engineers create convolutional neural networks in a bigger proportion than DS.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q18_Part_1'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='General purpose image/video tools (PIL, cv2, skimage, etc)', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='General purpose image/video tools',title_x=0.5)
fig.show()

ML Engineers use general purpose image/video tools in a bigger proportion than DS.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q18_Part_2'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Image segmentation methods (U-Net, Mask R-CNN, etc)', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Image segmentation methods',title_x=0.5)
fig.show()

Image segmentation methods are more used by ML engineers than by DS.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q14_Part_4'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Ggplot / ggplot2', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Ggplot / ggplot2',title_x=0.5)
fig.show()

DS use Ggplot in a bigger proportion than ML Engineers.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q17_Part_3'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Gradient Boosting Machines (xgboost, lightgbm, etc)', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Gradient Boosting Machines',title_x=0.5)
fig.show()

DS use gradient boosting machines in a bigger proportion than ML Engineers.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q31_A_Part_2'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Microsoft Power BI', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Microsoft Power BI',title_x=0.5)
fig.show()

As we noticed earlier Microsoft Power BI is used more by DS.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q18_Part_5'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Generative Networks (GAN, VAE, etc)', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Generative Networks',title_x=0.5)
fig.show()

Already predictable, ML Engineers create more generative networks than DS.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q17_Part_2'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Decision Trees or Random Forests', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Decision Trees or Random Forests',title_x=0.5)
fig.show()

DS use more than ML Engineers Decision Tree or Random Forest Classifiers.

In [ ]:
#Q37_Part_4
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q37_Part_4'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='DataCamp', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='DataCamp',title_x=0.5)
fig.show()

DS learn from DataCamp more than ML Engineers.

In [ ]:
#Q12_Part_1
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q12_Part_1'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='GPUs', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='GPUs usage',title_x=0.5)
fig.show()

ML Engineers use GPUs more than DS.

In [ ]:
#Q29_A_Part_8
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q29_A_Part_8'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Microsoft SQL Server', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Microsoft SQL Server usage',title_x=0.5)
fig.show()

DS use Microsoft SQL Server more than ML Engineers.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q2'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)
fig = go.Figure(data=[
    go.Bar(name='Data Scientist',x=ax.columns,y=ax.values[0,:]),
    go.Bar(name='ML Engineers', x=ax.columns, y=ax.values[1,:])
])
fig.update_layout(barmode='group')
fig.update_layout(title_text='Gender Distribution',title_x=0.5)
fig.show()

In [ ]:
#Q29_A_Part_8
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q2'].fillna('unfilled'),values='Q5',normalize='index',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Man', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='Nonbinary', x=ax.index, y=ax.values[:,1]),
    go.Bar(name='Prefer not to say', x=ax.index, y=ax.values[:,2]),
    go.Bar(name='Prefer to self-describe', x=ax.index, y=ax.values[:,3]),
    go.Bar(name='Woman', x=ax.index, y=ax.values[:,4])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Gender Distribution stacked',title_x=0.5)
fig.show()

When we analyse DS and ML Engineers from the age point of view, we can say that the values are comparable. There are some slightly differences, but I consider that more data is necessary in order to infer that ML engineers or DS are represented, definetely, by men.

In [ ]:
ax= pd.crosstab(data['Q5'], data['Q38'],normalize='index',values='Q5',aggfunc='count')
display(ax)
fig = go.Figure(data=[
    go.Bar(name='Data Scientist',x=ax.columns,y=ax.values[0,:]),
    go.Bar(name='ML Engineers', x=ax.columns, y=ax.values[1,:])
])
fig.update_layout(barmode='group')
fig.update_layout(title_text='Primary tool used to analyse data- percent',title_x=0.5)
fig.show()

DS use in a greater proportion than ML Engineers advanced statistical software. ML Engineers use in a greater proortion than DS basic statistical software. There is a slightly difference concerning the preference of ML Engineers for Cloud base data software (AWS,GCP, Azure).

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q17_Part_6'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Dense Neural Networks (MLPs, etc)', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Dense Neural Networks',title_x=0.5)
fig.show()

ML Engineers create more Dense Neural networks than DS.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q31_A_Part_5'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Tableau usage', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Tableau usage',title_x=0.5)
fig.show()

DS use Tableau more than ML engineers do.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q17_Part_8'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Generative Adversarial Networks', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Generative Adversarial Networks',title_x=0.5)
fig.show()

ML Engineers create more generative adversarial networks than DS, but not in a significant proportion. It is very possible to notice a positive trend of using generative adversarial networks by DS in the future.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q16_Part_7'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Xgboost', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Xgboost',title_x=0.5)
fig.show()

DS use more Xgboost than ML Engineers do.

In [ ]:
ax= pd.crosstab(data['Q5'], data['Q21'],normalize='index',values='Q5',aggfunc='count')
display(ax)
fig = go.Figure(data=[
    go.Bar(name='Data Scientist',x=ax.columns,y=ax.values[0,:]),
    go.Bar(name='ML Engineers', x=ax.columns, y=ax.values[1,:])
])
fig.update_layout(barmode='group')
fig.update_layout(title_text='Individuals responsible for data science workloads at work- percent',title_x=0.5)
fig.show()

ML Engineers work in workloads with few number of individuals, possible in smaller teams, than DS work.

In [ ]:
ax= pd.crosstab(data['Q5'], data['Q20'],normalize='index',values='Q5',aggfunc='count')
display(ax)
fig = go.Figure(data=[
    go.Bar(name='Data Scientist',x=ax.columns,y=ax.values[0,:]),
    go.Bar(name='ML Engineers', x=ax.columns, y=ax.values[1,:])
])
fig.update_layout(barmode='group')
fig.update_layout(title_text='Company size- percent',title_x=0.5)
fig.show()

ML Engineers work in companies that are smaller than those where DS work.

In [ ]:
ax= pd.crosstab(data['Q5'], data['Q8'],normalize='index',values='Q5',aggfunc='count')
display(ax)
fig = go.Figure(data=[
    go.Bar(name='Data Scientist',x=ax.columns,y=ax.values[0,:]),
    go.Bar(name='ML Engineers', x=ax.columns, y=ax.values[1,:])
])
fig.update_layout(barmode='group')
fig.update_layout(title_text='Recommended programming language for an aspiring DS- percent',title_x=0.5)
fig.show()

DS recommend R and SQL in bigger proportion than ML Engineers. Both DS and ML Engineers recommend Python, ML Engineers in a bigger proportion.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q33_A_Part_1'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Automated data augmentation (e.g. imgaug, albumentations)', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Automated data augmentation',title_x=0.5)
fig.show()

Automated data augmentation is used more by ML Engineers.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q16_Part_2'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='TensorFlow', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='TensorFlow usage',title_x=0.5)
fig.show()

TensorFlow is used more by ML Engineers. It is possible to notice in the future a positive trend of TensorFlow use by DS. 

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q17_Part_9'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Recurrent Neural Networks', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Recurrent Neural Networks',title_x=0.5)
fig.show()

Recurrent Neural Networks are created more by ML Engineers.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q19_Part_4'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Transformer language models (GPT-3, BERT, XLnet, etc)', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Transformer language models',title_x=0.5)
fig.show()

Transformer language models are used more by ML Engineers.

In [ ]:
ax= pd.crosstab(data['Q5'], data['Q22'],normalize='index',values='Q5',aggfunc='count')
display(ax)
fig = go.Figure(data=[
    go.Bar(name='Data Scientist',x=ax.columns,y=ax.values[0,:]),
    go.Bar(name='ML Engineers', x=ax.columns, y=ax.values[1,:])
])
fig.update_layout(barmode='group')
fig.update_layout(title_text='Current employer incorporate machine learning methods into their business- percent',title_x=0.5)
fig.show()

DS have well established ML methods at their workplace, in a greater proportion than ML Engineers.

DS and ML Engineerings common characteristics:
- they do not differ from the perspective of their number of years spent to code or programmining (Q6)
- they do not differ from the perspective of their preferences for hosted notebooks (Q10)
- they do not differ from the perspective of their specific preference for a particular computing platform (Q11)
- they do not differ from the perspective of how many times they used TPU (Q13)
- they do not differ from the point of view of the number of years since they use ML methods (Q15)
- they do not differ from the point of view of how much money have they (or their team) spent on machine learning and/or cloud computing services at home (or at work) in the past 5 years (Q25)
- they do not differ from the point of view of the cloud computing platforms or cloud computing products they use on a regular basis (Q26 and Q27)
- they do not differ from the point of view of the machine learning products they use on a regular basis (Q28)
- they do not differ from the point of view of the automated machine learning tools (or partial AutoML tools) they use on a regular basis (Q34)
- they do not differ from the point of view of where do they publicly share or deploy their data analysis or machine learning applications (Q36)
- they do not differ from the point of view of their preference for the favorite media sources that report on data science topics (Q39)


**As about the future....**

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q31_B_Part_5'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

fig = go.Figure(data=[
    go.Bar(name='Tableau in the next 2 years', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Willing to learn Tableau in the next 2 years',title_x=0.5)
fig.show()


DS are interested in investing time in learning Tableau in the next two years in a greater proportion than ML Engineers.

In [ ]:
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q31_B_OTHER'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)

#ax.plot(kind='bar', stacked=True, title = 'Other BI Tool to learn more in the next 2 years ', mark_right = True)
fig = go.Figure(data=[
    go.Bar(name='Other BI Tool in the next 2 years', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Willing to learn other BI Tool in the next 2 years',title_x=0.5)
fig.show()

DS and ML Engineers are quite content with BI tools they use. About their intention to learn any other BI tool (not specified in the survey) in the next two years, we can say that ML Engineers have, in a very small amount, greater expectations than DS in learning another BI tool.

In [ ]:
#Q33_B_Part_4
ax= pd.crosstab(data['Q5'].fillna('unfilled'), data['Q33_B_Part_4'].fillna('unfilled'),normalize='index',values='Q5',aggfunc='count')
display(ax)
fig = go.Figure(data=[
    go.Bar(name='Automated model architecture searches (e.g. darts, enas)', x=ax.index, y=ax.values[:,0]),
    go.Bar(name='unfilled', x=ax.index, y=ax.values[:,1])
])
fig.update_layout(barmode='stack')
fig.update_layout(title_text='Willing to find more about AutoML in the next 2 years',title_x=0.5)
fig.show()

DS and ML Engineers are interested in learning in the next 2 years more automated machine learning tools (or partial AutoML tools), ML Engineers in a bigger proportion.  What present interest in this field are....automated model architecture searches (e.g. darts, enas).

Surely, we will notice changes in DS and ML Engineers preferences in the next years. The future is yet to come.